In [1]:
import pandas as pd
import numpy as np
import io
import math
import random
import matplotlib.pyplot as plt
import librosa
import librosa.display
import cv2
import cmapy
import nlpaug
import nlpaug.augmenter.audio as naa
from scipy.signal import butter, lfilter
import torch
from tqdm import tqdm
import os
import magic
import augly.audio as audaugs
from IPython.display import display, Audio

In [2]:
audio_train=pd.read_csv('Temp_test_audio.csv')
audio_train.head(4)

,Unnamed: 0,Patient_ID,Disease,Recording_index,Chest_Location,Acquisition_Mode,Recording_equipment,Respiratory_cycles,Normal_cycles,Wheeze_cycles,Crackle_cycles,Both,Filename,Train/Test
0,217,136,NaN,1b1,Ar,sc,Meditron,9,9,0,0,0,136_1b1_Ar_sc_Meditron.txt,test
1,247,139,NaN,1b1,Al,sc,Litt3200,10,10,0,0,0,139_1b1_Al_sc_Litt3200.txt,test
2,248,139,NaN,1b1,Ar,sc,Litt3200,9,9,0,0,0,139_1b1_Ar_sc_Litt3200.txt,test
3,249,139,NaN,1b1,Ll,sc,Litt3200,8,7,0,0,1,139_1b1_Ll_sc_Litt3200.txt,test


In [3]:
sample_rate=4000
fs=30000
data_dir='AudioFiles2/'
desired_length=8
n_mels = 64
nfft = 256
hop = nfft//2
f_max = 2000
train_flag=1
lowcut=50
highcut=2000

In [4]:
def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a

def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = lfilter(b,a, data)
    return y

In [5]:
#slicing into breath cycles
def slice_data(start, end, raw_data, sample_rate):
    max_ind = len(raw_data) 
    start_ind = min(int(start * sample_rate), max_ind)
    end_ind = min(int(end * sample_rate), max_ind)
    y=butter_bandpass_filter(raw_data[start_ind: end_ind], lowcut, highcut, fs, order=5)
    return y

In [6]:
#getting label of each cycles
def get_label(crackle, wheeze):
    if crackle == 0 and wheeze == 0:
        return 0
    elif crackle == 1 and wheeze == 0:
        return 1
    elif crackle == 0 and wheeze == 1:
        return 2
    else:
        return 3

In [7]:
#getting breath cycle samples
def get_sound_samples(file_name, data_dir, sample_rate):
    sample_data = [file_name]
    data, rate = librosa.load(os.path.join(data_dir, file_name+'.wav'), sr=sample_rate)
    recording_annotations=pd.read_csv(data_dir+file_name+'.txt', sep='\t', header=None)
    
    for i in range(len(recording_annotations.index)):
        row = recording_annotations.loc[i]
        start = row[0]
        end = row[1]
        crackles = row[2]
        wheezes = row[3]
        audio_chunk = slice_data(start, end, data, rate)
        sample_data.append((audio_chunk, start,end, get_label(crackles, wheezes)))
    return sample_data

In [8]:
#creating list of breath cycles with respective labels
filenames=audio_train['Filename']
filenames_with_labels=[]
cycle_list = []
classwise_cycle_list = []
for idx, filename in tqdm(enumerate(filenames)):
            file_name=filename.split('.')[0]
            print(file_name+" going on "+str(idx)+" out of "+str(len(filenames)))
            data = get_sound_samples(file_name, data_dir, sample_rate)
            cycles_with_labels = [(d[0], d[3], file_name, cycle_idx, 0) for cycle_idx, d in enumerate(data[1:])]
            cycle_list.extend(cycles_with_labels)
            for cycle_idx, d in enumerate(cycles_with_labels):
                filenames_with_labels.append(file_name+'_'+str(d[3])+'_'+str(d[1]))
                classwise_cycle_list.append((d[0],d[1]))

0it [00:00, ?it/s]

136_1b1_Ar_sc_Meditron going on 0 out of 10


3it [00:06,  1.70s/it]

139_1b1_Al_sc_Litt3200 going on 1 out of 10
139_1b1_Ar_sc_Litt3200 going on 2 out of 10
139_1b1_Ll_sc_Litt3200 going on 3 out of 10
139_1b1_Lr_sc_Litt3200 going on 4 out of 10


6it [00:06,  1.47it/s]

139_1b1_Pl_sc_Litt3200 going on 5 out of 10
139_1b1_Pr_sc_Litt3200 going on 6 out of 10
143_1b1_Al_sc_Meditron going on 7 out of 10


8it [00:08,  1.33it/s]

144_1b1_Al_sc_Meditron going on 8 out of 10
144_1b1_Tc_sc_Meditron going on 9 out of 10


10it [00:12,  1.21s/it]


In [9]:
display(cycle_list[0][1])
len(cycle_list)

0

93

In [10]:
display(Audio(cycle_list[0][0], rate=sample_rate))

In [11]:
aug = audaugs.Compose([audaugs.AddBackgroundNoise(),audaugs.Clicks()])

In [12]:
aug_audio, sr = audaugs.pitch_shift(cycle_list[0][0], n_steps=10)

In [13]:
aug_audio

array([-0.00620337, -0.0096352 , -0.01904592, ..., -0.02846355,
       -0.02811167, -0.03490093])

In [14]:
print(aug_audio.shape, cycle_list[0][0].shape)

(8572,) (8572,)


In [15]:
cycle_list_aug=[]
for i in range(len(cycle_list)):
    if cycle_list[i][1]!=0:
        cycle_list_aug.append((cycle_list[i][0], cycle_list[i][1]))
        aug_audio, sr=audaugs.pitch_shift(cycle_list[i][0], n_steps=10)
        cycle_list_aug.append((aug_audio, cycle_list[i][1]))
    else:
        cycle_list_aug.append((cycle_list[i][0], cycle_list[i][1]))
    #print(i)
len(cycle_list_aug)

104

In [16]:
display(type(cycle_list_aug[0]), type(cycle_list[0]), cycle_list_aug[0][1], cycle_list[0])

tuple

tuple

0

(array([-5.36899030e-05, -5.17179863e-04, -2.44551756e-03, ...,
        -1.64040479e-01, -1.64190946e-01, -1.63977057e-01]),
 0,
 '136_1b1_Ar_sc_Meditron',
 0,
 0)

In [17]:
def padding(array, xx, yy):
    """
    :param array: numpy array
    :param xx: desired height
    :param yy: desired width
    :return: padded array
    """
    h = array.shape[0]
    w = array.shape[1]
    a = max((xx - h) // 2,0)
    aa = max(0,xx - a - h)
    b = max(0,(yy - w) // 2)
    bb = max(yy - b - w,0)
    return np.pad(array, pad_width=((a, aa), (b, bb)), mode='constant')

In [18]:
for idx, sample in enumerate(cycle_list_aug):
    print(type(sample[0]), sample[0])

<class 'numpy.ndarray'> [-5.36899030e-05 -5.17179863e-04 -2.44551756e-03 ... -1.64040479e-01
 -1.64190946e-01 -1.63977057e-01]
<class 'numpy.ndarray'> [-5.57448134e-06 -5.33574735e-05 -2.50344552e-04 ...  1.25706215e-01
  1.18222741e-01  1.11026220e-01]
<class 'numpy.ndarray'> [ 2.13204646e-05  2.05915492e-04  9.77183554e-04 ... -1.30819273e-01
 -1.26692058e-01 -1.22823776e-01]
<class 'numpy.ndarray'> [-2.18932458e-05 -2.11295211e-04 -1.00147003e-03 ... -5.95232474e-02
 -6.29295678e-02 -6.58988988e-02]
<class 'numpy.ndarray'> [4.38271136e-07 5.06742033e-06 2.88792401e-05 ... 1.31617094e-01
 1.77430281e-01 2.20164992e-01]
<class 'numpy.ndarray'> [ 5.20454228e-05  5.06544751e-04  2.42594353e-03 ... -1.38790330e-01
 -1.42331811e-01 -1.45404617e-01]
<class 'numpy.ndarray'> [ 1.52728380e-05  1.47131317e-04  6.95863159e-04 ... -1.68514520e-01
 -1.52203977e-01 -1.35541847e-01]
<class 'numpy.ndarray'> [-1.10370906e-04 -1.06606682e-03 -5.05868191e-03 ... -1.29059272e-01
 -1.30761183e-01 -1.3257

In [19]:
#creating sliced and padded data = audio_data
stft_data=[]
for idx, sample in enumerate(cycle_list_aug):
            print(str(idx)+" out of "+str(len(cycle_list_aug)))
            stft=librosa.stft(sample[0], n_fft=512, hop_length = 64)
            #print(stft.shape, sample[0].shape)
            stft_data.append(padding(stft, 257, 1011))

0 out of 104
1 out of 104
2 out of 104
3 out of 104
4 out of 104
5 out of 104
6 out of 104
7 out of 104
8 out of 104
9 out of 104
10 out of 104
11 out of 104
12 out of 104
13 out of 104
14 out of 104
15 out of 104
16 out of 104
17 out of 104
18 out of 104
19 out of 104
20 out of 104
21 out of 104
22 out of 104
23 out of 104
24 out of 104
25 out of 104
26 out of 104
27 out of 104
28 out of 104
29 out of 104
30 out of 104
31 out of 104
32 out of 104
33 out of 104
34 out of 104
35 out of 104
36 out of 104
37 out of 104
38 out of 104
39 out of 104
40 out of 104
41 out of 104
42 out of 104
43 out of 104
44 out of 104
45 out of 104
46 out of 104
47 out of 104
48 out of 104
49 out of 104
50 out of 104
51 out of 104
52 out of 104
53 out of 104
54 out of 104
55 out of 104
56 out of 104
57 out of 104
58 out of 104
59 out of 104
60 out of 104
61 out of 104
62 out of 104
63 out of 104
64 out of 104
65 out of 104
66 out of 104
67 out of 104
68 out of 104
69 out of 104
70 out of 104
71 out of 104
72

In [20]:
train_dataset=[]
for index in range(len(stft_data)):
    stft=stft_data[index]
    label = cycle_list_aug[index][1]
    train_dataset.append((stft, label))

In [21]:
X_train=[]
y_train=[]
for i in range(len(train_dataset)):
    X_train.append(train_dataset[i][0])
    y_train.append(train_dataset[i][1])  
    
np.save('X_test_STFT_small_aug', np.concatenate(X_train, axis=0).reshape(len(train_dataset), 257, 1011, 1))
#np.save('X_test_STFT_small', X_train)
np.save('y_test_STFT_small_aug', y_train)